In [1]:
import numpy as np
import pandas as pd
import sklearn
from pathlib import Path
import matplotlib.pyplot as plt
import re
from utils import *

pd.set_option('future.no_silent_downcasting', True)

In [2]:
path_data = Path("../data")

# Read tractors data

In [3]:
tractor_data = []
for folder in (path_data / "исходные данные от оператора").iterdir():
    for file in folder.iterdir():
        name = file.parent.name
        print(name, end=" ")
        data = pd.read_csv(file, sep=";", low_memory=False)
        data["tractor_type"] = name
        tractor_data.append(data)

tractor_data = pd.concat(tractor_data, axis=0, ignore_index=True)

С2177 С2177 С2627 7226ом61 С2627 7226ом61 С2627 7226ом61 Р3106 Р3106 С0790 С0790 022C4097 022C4097 C1969 C1969 023Т0255 023Т0255 С2586 С2586 С1067 С1067 С1067 С1067 023Т0719 023Т0719 023T0569 023T0569 023Т0049 023Т0049 С1178 С1178 С0873 С0873 

In [4]:
tractor_data.head()

,Дата и время,"Полож.пед.акселер.,%","Нагрузка на двигатель, %","Давл.масла двиг.,кПа","Темп.масла двиг.,°С","Обор.двиг.,об/мин","Значение счетчика моточасов, час:мин",Сост.пед.сцепл.,iButton2,КПП. Температура масла,...,Холодный старт (spn3871),"Крутящий момент (spn513), Нм","Положение рейки ТНВД (spn51), %","Расход топлива (spn183), л/ч","ДВС. Температура наддувочного воздуха, °С","Давление наддувочного воздуха двигателя (spn106), кПа",Текущая передача (spn523),"Температура масла гидравлики (spn5536), С",Педаль слива (spn598),tractor_type
0,01/05/2023 00:00:09,"0,0",-,160,"-273,000","699,625",3190:18,Отпущ.,NaN,-273,...,-,-,-,-,-,-,-,-,-,С2177
1,01/05/2023 00:00:39,"0,0",-,160,"-273,000","700,125",3190:24,Отпущ.,NaN,-273,...,-,-,-,-,-,-,-,-,-,С2177
2,01/05/2023 00:01:09,"0,0",-,172,"-273,000","701,125",3190:24,Отпущ.,NaN,-273,...,-,-,-,-,-,-,-,-,-,С2177
3,01/05/2023 00:01:39,"0,0",-,172,"-273,000","699,000",3190:24,Отпущ.,NaN,-273,...,-,-,-,-,-,-,-,-,-,С2177
4,01/05/2023 00:02:09,"0,0",-,172,"-273,000","701,000",3190:24,Отпущ.,NaN,-273,...,-,-,-,-,-,-,-,-,-,С2177


In [5]:
tractor_data.drop(cols_to_remove, axis=1, inplace=True)

In [6]:
print(tractor_data.shape)

fill_space_with_nans(tractor_data)
tractor_data = remove_null_ros(tractor_data, additional_col="tractor_type")

print(tractor_data.shape)

(3156090, 12)
(2666861, 12)


In [7]:
brake_int_cols = ["Полож.пед.акселер.,%", "Обор.двиг.,об/мин"]
for col in brake_int_cols:
    tractor_data[col] = tractor_data[col].str.replace(",", ".")

In [8]:
for num_col in numeric_cols:
    tractor_data[num_col] = tractor_data[num_col].astype(float)

In [9]:
tractor_data["Дата и время"] = pd.to_datetime(tractor_data["Дата и время"], format='%d/%m/%Y %H:%M:%S', dayfirst=True)

In [10]:
pd.to_pickle(tractor_data, path_data / "operator_with_nan.pickle")

In [11]:
tractor_data.bfill(inplace=True)
tractor_data.ffill(inplace=True)

In [12]:
tractor_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2666861 entries, 0 to 3156086
Data columns (total 12 columns):
 #   Column                                  Dtype         
---  ------                                  -----         
 0   Полож.пед.акселер.,%                    float64       
 1   Давл.масла двиг.,кПа                    float64       
 2   Обор.двиг.,об/мин                       float64       
 3   КПП. Давление масла в системе смазки    float64       
 4   Скорость                                float64       
 5   ДВС. Давление смазки                    float64       
 6   ДВС. Температура охлаждающей жидкости   float64       
 7   Давление в пневмостистеме (spn46), кПа  float64       
 8   Электросистема. Напряжение              float64       
 9   ДВС. Частота вращения коленчатого вала  float64       
 10  Дата и время                            datetime64[ns]
 11  tractor_type                            object        
dtypes: datetime64[ns](1), float64(10), object(1)
me

In [13]:
tractor_data.describe()

,"Полож.пед.акселер.,%","Давл.масла двиг.,кПа","Обор.двиг.,об/мин",КПП. Давление масла в системе смазки,Скорость,ДВС. Давление смазки,ДВС. Температура охлаждающей жидкости,"Давление в пневмостистеме (spn46), кПа",Электросистема. Напряжение,ДВС. Частота вращения коленчатого вала,Дата и время
count,2.666861e+06,2.666861e+06,2.666861e+06,2.666861e+06,2.666861e+06,2.666861e+06,2.666861e+06,2.666861e+06,2.666861e+06,2.666861e+06,2666861
mean,3.358586e+01,4.163493e+02,1.579796e+03,1.183169e+03,5.586458e+00,4.163584e+02,7.455521e+01,7.625279e+02,2.742555e+01,1.579799e+03,2023-07-12 00:23:28.743797248
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.130000e+01,0.000000e+00,2023-05-01 00:00:09
25%,0.000000e+00,3.600000e+02,1.234000e+03,1.200000e+03,0.000000e+00,3.600000e+02,7.300000e+01,7.440000e+02,2.770000e+01,1.234125e+03,2023-06-19 19:58:23
50%,0.000000e+00,4.680000e+02,1.885500e+03,1.216000e+03,3.500000e+00,4.680000e+02,7.500000e+01,7.600000e+02,2.795000e+01,1.885500e+03,2023-07-15 13:50:04
75%,9.480000e+01,5.080000e+02,1.902250e+03,1.264000e+03,1.030000e+01,5.080000e+02,7.700000e+01,7.840000e+02,2.810000e+01,1.902250e+03,2023-08-08 16:52:00
max,1.000000e+02,9.880000e+02,2.340750e+03,1.392000e+03,3.860000e+01,9.880000e+02,1.150000e+02,8.480000e+02,3.060000e+01,2.340750e+03,2023-10-13 09:28:18
std,4.301128e+01,1.255061e+02,4.825573e+02,2.034957e+02,6.420454e+00,1.254904e+02,8.237140e+00,4.997847e+01,2.543912e+00,4.825617e+02,NaN


In [14]:
tractor_data.isna().sum()

Полож.пед.акселер.,%                      0
Давл.масла двиг.,кПа                      0
Обор.двиг.,об/мин                         0
КПП. Давление масла в системе смазки      0
Скорость                                  0
ДВС. Давление смазки                      0
ДВС. Температура охлаждающей жидкости     0
Давление в пневмостистеме (spn46), кПа    0
Электросистема. Напряжение                0
ДВС. Частота вращения коленчатого вала    0
Дата и время                              0
tractor_type                              0
dtype: int64

In [15]:
pd.to_pickle(tractor_data, path_data / "operator_clear.pickle")